In [1]:
%cd ..
%pip install -e .

/home/sandor/dtu/2021-22-spring/advanced_machine_learning/p-reg
Obtaining file:///home/sandor/dtu/2021-22-spring/advanced_machine_learning/p-reg
  Preparing metadata (setup.py) ... done
  Attempting uninstall: src
    Found existing installation: src 0.1.0
    Uninstalling src-0.1.0:
      Successfully uninstalled src-0.1.0
  Running setup.py develop for src
Note: you may need to restart the kernel to use updated packages.


In [2]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tqdm import tqdm

from src.models.train_model import train_with_loss
from src.models.train_model import random_splits

from src.models.gcn import GCN_fix_2layer

from src.models.reg import make_preg_ce_ce
from src.models.reg import make_preg_ce_ce_alt
from src.models.reg import make_preg_loss
from src.models.reg import make_abduls_preg_loss
from src.models.reg import make_lap_loss_ce
from src.models.reg import compute_a_hat

from src.models.evaluate_model import evaluate1

import torch
import torch.nn.functional as F

from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T



In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
dataset = Planetoid(root='data/Planetoid', name='Cora', transform=T.NormalizeFeatures())
data = dataset[0].to(device)

A_hat = compute_a_hat(data)

# data = random_splits(data, 50, 20)
data.reg_mask = torch.ones_like(data.train_mask, dtype=torch.bool)


In [4]:
print('-------------------------------------------------------------')
print(f'train size: {data.train_mask.sum()}')
print(f'val size: {data.val_mask.sum()}')
print(f'test size: {data.test_mask.sum()}')
print('-------------------------------------------------------------')

metrics = []

for mu in [0, .1, .2, .3, .4, .5, .6, .7, .8, .9, 1.]:
    torch.manual_seed(1)
    random.seed(1)
    L_cls = lambda x, y: F.cross_entropy(x, y, reduction='sum')
    L_preg = lambda x, y: F.cross_entropy(x, y, reduction='sum')
    loss_fn = make_preg_loss(L_cls, L_preg, mu, A_hat)
    
    model = GCN_fix_2layer()

    model = train_with_loss(model, data, loss_fn, num_epochs=200)

    train_acc, val_acc, test_acc = evaluate1(model, data)
    metrics.append({'mu': mu, 'train_acc': np.round(train_acc,4), 'val_acc': np.round(val_acc,4), 'test_acc': np.round(test_acc,4)})
    print(metrics[-1])


-------------------------------------------------------------
train size: 140
val size: 500
test size: 1000
-------------------------------------------------------------
{'mu': 0, 'train_acc': 0.8286, 'val_acc': 0.59, 'test_acc': 0.575}
{'mu': 0.1, 'train_acc': 0.5714, 'val_acc': 0.426, 'test_acc': 0.409}
{'mu': 0.2, 'train_acc': 0.2857, 'val_acc': 0.114, 'test_acc': 0.133}
{'mu': 0.3, 'train_acc': 0.1429, 'val_acc': 0.058, 'test_acc': 0.064}
{'mu': 0.4, 'train_acc': 0.1429, 'val_acc': 0.072, 'test_acc': 0.091}


KeyboardInterrupt: 